In [42]:
import tensorflow as tf
#from tqdm import tdqm
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1DTranspose, Conv1D, Reshape, Input, ReLU, UpSampling1D, LeakyReLU
import json
import time
tf.compat.v1.enable_eager_execution

<function tensorflow.python.framework.ops.enable_eager_execution(config=None, device_policy=None, execution_mode=None)>

In [43]:
def makeGen():
    model = tf.keras.Sequential()
    model.add(Input(shape = (100,)))
    model.add(Dense(256, use_bias= True))
    model.add(Reshape((16,16)))
    model.add(ReLU())
    print("2",model.output_shape)

    model.add(Conv1DTranspose(64, kernel_size = 25,strides = 4, use_bias = True,padding ='same'))
    model.add(ReLU())
    print("3",model.output_shape)
    
    model.add(Conv1DTranspose(4, kernel_size = 25,strides = 4, use_bias = True,padding ='same'))
    model.add(ReLU())
    print("4",model.output_shape)

    model.add(Conv1DTranspose(2, kernel_size = 25,strides = 4, use_bias = True,padding ='same'))
    model.add(ReLU())
    print("5",model.output_shape)

    model.add(Conv1DTranspose(1, kernel_size = 25,strides = 4, use_bias = True,padding ='same'))
    model.add(ReLU())
    print("6",model.output_shape)

    model.add(Conv1DTranspose(1, kernel_size = 25,strides = 4, use_bias = True,padding ='same'))
    model.add(ReLU())
    print("7",model.output_shape)

    model.add(Activation('tanh'))
    return model
makeGen()

2 (None, 16, 16)
3 (None, 64, 64)
4 (None, 256, 4)
5 (None, 1024, 2)
6 (None, 4096, 1)
7 (None, 16384, 1)


In [44]:
def makeDisc():
    model = tf.keras.Sequential()
    model.add(Input(shape = (16384,1)))
    print("0",model.output_shape)

    model.add(Conv1D(1,25,strides = 4, use_bias = True, padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    print("0",model.output_shape)
    model.add(Conv1D(2,25,strides = 4, use_bias = True, padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    print("0",model.output_shape)
    model.add(Conv1D(4,25,strides = 4, use_bias = True, padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    print("0",model.output_shape)
    model.add(Conv1D(8,25,strides = 4, use_bias = True, padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    print("0",model.output_shape)
    model.add(Conv1D(16,25,strides = 4, use_bias = True, padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    print("0",model.output_shape)
    model.add(Reshape((256,)))
    print("0",model.output_shape)
    model.add(Dense(1))
    print("0",model.output_shape)
    return model
makeDisc()


0 (None, 16384, 1)
0 (None, 4096, 1)
0 (None, 1024, 2)
0 (None, 256, 4)
0 (None, 64, 8)
0 (None, 16, 16)
0 (None, 256)
0 (None, 1)


In [45]:
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
    X = np.array(data["snares"])
    return X.astype(dtype = np.float32)

In [99]:
def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)

def discriminator_loss(real_sample, fake_sample):
    #print(real_sample, fake_sample)
    real_loss = tf.reduce_mean(real_sample)
    fake_loss = tf.reduce_mean(fake_sample)
    #print('real_loss')
    #print(real_loss)
    #print('fake_loss')
    #print(fake_loss)
    loss = fake_loss - real_loss
    #print('loss')
    #print(loss.shape)
    return loss

In [100]:
def gradient_penalty(batch_size, real_samples, fake_samples):
    alpha = tf.random.uniform(shape = [batch_size, 1,1], minval = 0., maxval = 1.)
    #print(real_samples.shape)
    #print(fake_samples.shape)
    
    diff = fake_samples - real_samples
    interp = real_samples + (alpha * diff)
    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interp)
        pred = discriminator(interp, training = True)
    gradients = gp_tape.gradient(pred, [interp])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis  = [1,2]))
    return tf.reduce_mean((norm-1.0) ** 2)



In [106]:
@tf.function
def train_step(real_samples, dpg = 5):
    batch_size = 64
    for i in range(dpg):
        random_latent_vectors = tf.random.normal(shape = (batch_size, 100))
        with tf.GradientTape() as tape:
            fake_samples = generator(random_latent_vectors, training = True)
            fake_logits = discriminator(fake_samples, training = True)
            real_logits = discriminator(real_samples, training = True)
            d_loss = discriminator_loss(real_logits, fake_logits)
            
            gp = gradient_penalty(batch_size, real_samples, fake_samples)
            d_loss = d_loss + (gp * 10)
            
        d_gradients = tape.gradient(d_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(d_gradients, discriminator.trainable_variables))
    

    random_latent_vectors = tf.random.normal(shape = (batch_size, 100))
    with tf.GradientTape() as tape:
        gen_samples = generator(random_latent_vectors, training = True)
        gen_logits = discriminator(gen_samples, training = True)
        g_loss = generator_loss(gen_logits)

    g_gradients = tape.gradient(g_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(g_gradients, generator.trainable_variables))
    print("Discriminator Loss:")
    tf.print(d_loss, output_stream=sys.stderr)
    print(" Loss:")
    tf.print(g_loss, output_stream=sys.stderr)
    return {"d_loss":d_loss, "g_loss": g_loss}




In [107]:
def train(dataset, epochs):
    for epoch in range(epochs):
        print("epoch #", epoch)
        for batch in dataset:
            train_step(batch)    

In [26]:
X = load_data("data.json")


#train(X, 300)


In [108]:
#print(X.dtype)
generator = makeGen()
discriminator = makeDisc()

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5, beta_2=0.9)
train_dataset == 



2 (None, 16, 16)
3 (None, 64, 64)
4 (None, 256, 4)
5 (None, 1024, 2)
6 (None, 4096, 1)
7 (None, 16384, 1)
0 (None, 16384, 1)
0 (None, 4096, 1)
0 (None, 1024, 2)
0 (None, 256, 4)
0 (None, 64, 8)
0 (None, 16, 16)
0 (None, 256)
0 (None, 1)


In [109]:
from scipy.io.wavfile import write as wavwrite

#print(X.shape)
#print(np.squeeze(X, axis = 3).shape)
##print(X)
squeezed = np.squeeze(X,axis = 2)
print(squeezed[0])
wavwrite("first.wav", 16000, squeezed[0])

[[0.00070278]
 [0.21556064]
 [0.3303949 ]
 ...
 [0.        ]
 [0.        ]
 [0.        ]]


In [110]:
seed = tf.random.normal([1, 100])

In [21]:
pred = generator(seed, training = False)

In [23]:
pred

<tf.Tensor: shape=(1, 16384, 1), dtype=float32, numpy=
array([[[0.00057547],
        [0.        ],
        [0.00034705],
        ...,
        [0.00072842],
        [0.        ],
        [0.00035185]]], dtype=float32)>

In [25]:
from scipy.io.wavfile import write as wavwrite


In [27]:
print(pred.shape)
pred = np.squeeze(pred.numpy(), axis =2)
print(pred.shape)


(1, 16384, 1)
(1, 16384)


In [30]:
wavwrite("preview2.wav", 16000, pred.T)

In [33]:
print(tf.__version__)

2.3.0
